In [1]:
# Program: E-Field Simulator.ipynb
# Purpose: Simulates the electric field generated by a charged ring, disk, or parallel-plate capacitor at a given observation point
# Author:  Tobias Safie - tks57@drexel.edu
# Date:    March 12, 2025
from vpython import *

# Constants
k = 9e9
scale = 1e-2
N = 30
Q = 1e-6
dtheta = 2*pi/N
dQ = Q/N
R = 2
s = 5

# Set scene
scene.width = 800
scene.height = 600
scene.background = color.white
axes = curve(pos=([(3,0,0),(-4,0,0),(-3,0,0),(-3,-4,0),(-3,4,0),(-3,0,0),(-3,0,-3),(-3,0,3)]), color=color.black)

# Define observation point
obs = vector(3,1.1,0)

# Use the mouse to create a new observation point
def mouseClick(evt):
    global obs
    obs = vec(evt.pos.x, evt.pos.y, 0)  # Project onto the x-y plane (set z = 0)
    print(f"Observation point updated to: {obs}")

# Bind mouse click event
scene.bind("click", mouseClick)

# Create buttons
# Button callback functions
def showRing():
    ERing = findEFieldAtPoint(obs, vec(-3, 0, 0), R, Q)
    print("Electric Field from Ring:", ERing)

def showDisk():
    EDisk = findEFieldDisk(obs, vec(-3, 0, 0), R, Q)
    print("Electric Field from Disk:", EDisk)

def showCapacitor():
    ECapacitor = findEFieldCapacitor(obs, vec(-3, 0, 0), R, Q, s)
    print("Electric Field from Capacitor:", ECapacitor)

# Create buttons
button(text="Ring", bind=showRing)
button(text="Disk", bind=showDisk)
button(text="Capacitor", bind=showCapacitor)

E=vector(0,0,0)
dE=vector(0,0,0)

# Iterate thru slices summing each component of the E field
# Sample code
def findEField():
    for n in range(N):
        rate(N/10)
        slice.pos=ri.pos+vector(0,R*sin(n*dtheta),R*cos(n*dtheta))
        r=obs-slice.pos
        ra.pos=slice.pos
        ra.axis=r
        dE=k*dQ*norm(r)/mag(r)**2
        E+=dE
        Ea.axis=E*scale

# Part A
def findEFieldAtPoint(obs, ringPos, R, Q, existingE=vec(0,0,0)):
    # Update E and dQ with parameter values
    E = existingE
    dQ = Q / N

    # Create objects
    ri = ring(pos=vec(-3,0,0), axis=vec(1,0,0), radius=R, color=color.red, thickness=0.05)
    slice = sphere(pos=ri.pos+vector(0,0,R),radius=0.1,color=color.cyan)
    r=obs-slice.pos
    ra=arrow(pos=slice.pos,axis=r,color=color.green,shaftwidth=0.1)
    Ea=arrow(pos=obs,axis=vec(0,0,0),color=vec(1,0.8,0))

    
    # Sum the slices to find the E field
    for n in range(N):
        rate(N/10)
        theta = n * dtheta

        slice.pos=ri.pos+vector(0,R*sin(n*dtheta),R*cos(n*dtheta))
        r=obs-slice.pos
        ra.pos=slice.pos
        ra.axis=r
        dE=k*dQ*norm(r)/mag(r)**2
        E+=dE
        Ea.axis=E*scale

        # Calculate r vector
        r = obs - slice.pos
        if mag(r) == 0:  # Avoid division by zero
            continue

        # Compute Efield
        dE = k * dQ * norm(r) / mag(r)**2
        E += dE 

    # Return final Efield at observation point
    return E

# Part B
def findEFieldDisk(obs, diskPos, R, Q, existingE=vec(0,0,0)):
    # Update E with parameter values
    E = existingE
    dr = 0.05 # Set a value for the differential

    # Create the disk
    disk = cylinder(pos=vec(-3, 0, 0), axis=vec(1, 0, 0), radius=R, length=0.1, color=color.red)

    # Loop thru rings from 0 to R
    r = dr
    while r <= R:
        # Calculate charge for this ring
        dQ = Q * (2 * r * dr / R)

        # Calculate Efield given by this differential ring
        E += findEFieldAtPoint(obs, diskPos, r, dQ, E)

        # Update r
        r += dr

    return E

# Part C
def findEFieldCapacitor(obs, platePos, R, Q, s, existingE=vec(0,0,0)):
    # Position of the two plates
    plate1Pos = platePos + vec(0, 0, 0)  # Upper plate (+Q)
    plate2Pos = platePos - vec(-s, 0, 0)  # Lower plate (-Q)

    plate1 = cylinder(pos=plate1Pos, axis=vec(1, 0, 0), radius=R, length=0.1, color=color.red)
    plate2 = cylinder(pos=plate2Pos, axis=vec(1, 0, 0), radius=R, length=0.1, color=color.red)

    # Compute fields from both disks
    E1 = findEFieldDisk(obs, plate1Pos, R, Q, existingE)  # +Q Plate
    E2 = findEFieldDisk(obs, plate2Pos, R, -Q, existingE) # -Q Plate

    # Total for net E
    E = E1 + E2

    return E

<IPython.core.display.Javascript object>